In [ ]:
import pandas as pd

In [ ]:
dataset_train1 = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv', index_col='id')
dataset_test1 = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv', index_col='id')
y = dataset_train1.target
y = pd.DataFrame(y)
dataset_train = dataset_train1.drop(['target'], axis=1)

reduce memory usage

In [ ]:
for col in dataset_train.columns:
    if dataset_train[col].dtype == 'float64':
        dataset_train[col] = dataset_train[col].astype('float32')
        dataset_test1[col] = dataset_test1[col].astype('float32')
    if dataset_train[col].dtype == 'int64':
        dataset_train[col] = dataset_train[col].astype('int8')
        dataset_test1[col] = dataset_test1[col].astype('int8') #

In [ ]:
#dataset_train['nan_count'] = dataset_train.isnull().sum(axis=1)

In [ ]:
#dataset_test1['nan_count'] = dataset_test1.isnull().sum(axis=1)

In [ ]:
import tensorflow as tf

In [ ]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
from sklearn.preprocessing import QuantileTransformer, KBinsDiscretizer, PowerTransformer

from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
dataset_train = imputer.fit_transform(dataset_train)
dataset_test = imputer.transform(dataset_test1)

#pt = PowerTransformer()
#dataset_train = pt.fit_transform(dataset_train)
#dataset_test = pt.transform(dataset_test)

qt = QuantileTransformer(n_quantiles=300, output_distribution='uniform')
dataset_train = qt.fit_transform(dataset_train)
dataset_test = qt.transform(dataset_test1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler()
dataset_train_sc = x_scaler.fit_transform(dataset_train)
dataset_test_sc = x_scaler.transform(dataset_test)

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
#from tensorflow.keras.engine.input_layer import Input
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import BatchNormalization

In [ ]:
def model_builder(lr):
    """Модель нейронной сети для агента"""
    inputA = keras.Input(shape=(285))
    line = Reshape((285,1))(inputA)
    line = Conv1D(filters=64, kernel_size=2, padding='same', activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.1)(line)

    line = Conv1D(filters=128, kernel_size=2, activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.3)(line)
    
    line = MaxPooling1D(pool_size=2)(line)
    
    line = Conv1D(filters=256, kernel_size=2, activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.5)(line)
    
    line = MaxPooling1D(pool_size=2)(line)

    line = Flatten()(line)
    line = Dense(64, activation='relu')(line)
    line = Dropout(0.5)(line)
    #line = Dense(64, activation='relu')(line)
    outputA = Dense(units=1, activation="sigmoid", kernel_regularizer=keras.regularizers.l1(0.01))(line)
    model = Model(inputs=inputA, outputs=outputA)
    #model = keras.models.load_model('models/model2')
    model.compile(loss = 'binary_crossentropy', optimizer = Adam(lr=lr), metrics=[tf.keras.metrics.AUC(name='auc'), 'binary_accuracy'],)
    return model

In [ ]:
lr=0.0001
with strategy.scope():
    model = model_builder(lr)

In [ ]:
model.summary()

In [ ]:
checkpoint_filepath = 'pseudo_best.h5'
save_model_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_auc',
    mode='max',
    verbose=1,
    save_best_only=True)

repeated pseudo training

In [ ]:
#model = load_model('../input/tps1021/best_85221.h5')

preds = model.predict(dataset_test_sc)

for i in range(len(preds)):
    preds[i] = 1 if preds[i] >= 0.5 else 0

In [ ]:
N_split = int(0.05 * len(dataset_train_sc))
dataset_sc_TRAIN = dataset_train_sc[:-N_split, :]
dataset_sc_VAL = dataset_train_sc[-N_split:, :]
y_TRAIN = y[:-N_split]
y_VAL = y[-N_split:]

import numpy as np
big_dataset_X = np.concatenate([dataset_sc_TRAIN, dataset_test_sc], axis=0)
len(big_dataset_X)

big_dataset_y = np.concatenate([y_TRAIN, preds], axis=0)
len(big_dataset_y)

N = len(preds)/(len(preds)+len(y_TRAIN))
N

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_auc', factor=0.2, patience=1, min_lr=0.0001, verbose=1, mode='max')

pseudo training

EPOCHS = 50
#EPOCHS = 1
model.fit(
    big_dataset_X, big_dataset_y,
    validation_data=(dataset_sc_VAL, y_VAL), epochs=EPOCHS, callbacks=[save_model_callback, reduce_lr], batch_size=512, shuffle=True)

real training

In [ ]:
EPOCHS = 200
#EPOCHS = 1
model.fit(
    dataset_sc_TRAIN, y_TRAIN,
    validation_data=(dataset_sc_VAL, y_VAL), epochs=EPOCHS, callbacks=[save_model_callback, reduce_lr], batch_size=4096, shuffle=True)

repeat few times 2 cells above

In [ ]:
#model = load_model('pseudo_best.h5')

In [ ]:
preds = model.predict(dataset_test_sc)
output = pd.DataFrame({'Id': dataset_test1.index,'target': preds[:,0]})
path = 'sample_submission.csv'
output.to_csv(path, index=False)
output 